In [1]:
# Import directories
import duckdb
import pandas as pd

print("Imported")

Imported


In [2]:
# Import dataframes
df = pd.read_csv('../data/01-raw/flights.csv',  low_memory=False)
df_airline = pd.read_csv('../data/01-raw/airlines.csv')
df_cancellation_codes = pd.read_csv('../data/01-raw/cancellation_codes.csv')

df1 = df.copy()
df2 = df_airline.copy()
df3 = df_cancellation_codes.copy()
print("Dataframes created.")

Dataframes created.


Question 1a: How does the overall flight volume vary by month?

In [3]:
# SQL Statement
df_ans = duckdb.sql("""
            SELECT MONTH as Month, monthname(make_date(2015, MONTH, ANY_VALUE(DAY))) as Month_Name, COUNT(FLIGHT_NUMBER) AS Flight_Volume,
            LAG(Flight_Volume) OVER (ORDER BY Month) AS old,
            (Flight_Volume - old)*100 / old AS Percentage_Change,
            Flight_Volume - old as Absolute_Change
            FROM df 
            GROUP BY Month
            ORDER BY Month ASC""").df()
# Drop the old axis, unneeded
df_ans.drop('old', axis = 1, inplace = True)
print("Overall flight volume, by month:")
print(df_ans)
print(f"Standard Deviation of Percentage Change: {df_ans['Percentage_Change'].std()}%\nStandard Deviation of Absolute Change: {df_ans['Absolute_Change'].std()}")

Overall flight volume, by month:
    Month Month_Name  Flight_Volume  Percentage_Change  Absolute_Change
0       1    January         469968                NaN             <NA>
1       2   February         429191          -8.676548           -40777
2       3      March         504312          17.502930            75121
3       4      April         485151          -3.799434           -19161
4       5        May         496993           2.440890            11842
5       6       June         503897           1.389154             6904
6       7       July         520718           3.338182            16821
7       8     August         510536          -1.955377           -10182
8       9  September         464946          -8.929831           -45590
9      10    October         486165           4.563756            21219
10     11   November         467972          -3.742145           -18193
11     12   December         479230           2.405699            11258
Standard Deviation of Percentag

Question 1b: By day of week?

In [4]:
# SQL Statement
df_ans = duckdb.sql("""
        WITH DATE_DF AS (
                SELECT dayname(make_date(2015, MONTH, DAY)) as Date, 
                DAY_OF_WEEK AS Day_Of_Week,
                COUNT(FLIGHT_NUMBER) AS Flight_Volume
                FROM df
                GROUP BY MONTH, DAY, DAY_OF_WEEK
                ORDER BY DAY
        ) SELECT Date as Day_Name, Day_Of_Week AS Day, AVG(Flight_Volume) as Average_Flight_Volume,
            LAG(Average_Flight_Volume) OVER (ORDER BY Day_Of_Week) as old,
            (Average_Flight_Volume - old) / old AS Percentage_Change,
            Average_Flight_Volume - old as Absolute_Change
            FROM DATE_DF
            GROUP BY Day_of_Week, Date
            ORDER BY Day_Of_Week
    """).df()
# Drop the old axis, unneeded
df_ans.drop('old', axis = 1, inplace = True)
print("Overall flight volume, by day of the week:")
print(df_ans)
print(f"Standard Deviation of Percentage Change: {df_ans['Percentage_Change'].std()}%\nStandard Deviation of Absolute Change: {df_ans['Absolute_Change'].std()}")

Overall flight volume, by day of the week:
    Day_Name  Day  Average_Flight_Volume  Percentage_Change  Absolute_Change
0     Monday    1           16645.057692                NaN              NaN
1    Tuesday    2           16242.307692          -0.024196      -402.750000
2  Wednesday    3           16459.557692           0.013376       217.250000
3   Thursday    4           16462.660377           0.000189         3.102685
4     Friday    5           16580.942308           0.007185       118.281930
5   Saturday    6           13472.019231          -0.187500     -3108.923077
6     Sunday    7           15726.230769           0.167325      2254.211538
Standard Deviation of Percentage Change: 0.1130283157767837%
Standard Deviation of Absolute Change: 1722.1829503643053


Question 2a: What percentage of flights in experienced a departure delay in 2015? 

In [5]:
# SQL Statement
df_ans = duckdb.sql("""
            WITH pivot_delay AS (
                        FROM df UNPIVOT INCLUDE NULLS (
                            Delay_Time
                            FOR Delay_Type IN (AIR_SYSTEM_DELAY, SECURITY_DELAY,
                            AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY)
                        )
            ) SELECT 'Overall_Flights' AS 'Delay_Type', COUNT(*) AS Flight_Volume, COUNT(NULLIF(Delay_Time, 0)) AS Delay_Count,
                    Delay_Count*100/Flight_Volume as Percentage_of_Delays,
                    AVG(Delay_Time) as Average_Delay_Time
                    
            FROM pivot_delay
        """).df()
print(df_ans)

        Delay_Type  Flight_Volume  Delay_Count  Percentage_of_Delays  \
0  Overall_Flights       29095395      1760001              6.049071   

   Average_Delay_Time  
0           11.782879  


Question 2b: Among those flights, what was the average delay time, in minutes?

In [6]:
# SQL Statement
df_ans = duckdb.sql("""
            WITH pivot_delay AS (
                        FROM df UNPIVOT INCLUDE NULLS (
                            Delay_Time
                            FOR Delay_Type IN (AIR_SYSTEM_DELAY, SECURITY_DELAY,
                            AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY)
                        )
            ) SELECT Delay_Type, COUNT(*) as Flight_Volume, COUNT(NULLIF(Delay_Time, 0)) as Delay_Count,
                        Delay_Count*100/Flight_Volume as Percentage_of_Delays,
                        AVG(Delay_Time) as Average_Delay_Time 
            FROM pivot_delay
            GROUP BY Delay_Type""").df()
print(df_ans)

            Delay_Type  Flight_Volume  Delay_Count  Percentage_of_Delays  \
0        WEATHER_DELAY        5819079        64716              1.112135   
1       SECURITY_DELAY        5819079         3484              0.059872   
2     AIR_SYSTEM_DELAY        5819079       564826              9.706450   
3  LATE_AIRCRAFT_DELAY        5819079       556953              9.571154   
4        AIRLINE_DELAY        5819079       570022              9.795743   

   Average_Delay_Time  
0            2.915290  
1            0.076154  
2           13.480568  
3           23.472838  
4           18.969547  


Question 3a: How does the % of delayed flights vary throughout the year?

In [7]:
# SQL Statement
df_ans = duckdb.sql("""
        SELECT MONTH as Month, monthname(make_date(2015, MONTH, ANY_VALUE(DAY))) as Month_Name, COUNT(FLIGHT_NUMBER) AS Flight_Volume,
        COUNT(AIR_SYSTEM_DELAY) AS Flight_Delayed, Flight_Delayed*100/Flight_Volume as Percentage_Of_Flights_Delayed,
        LAG(Flight_Volume) OVER (ORDER BY Month) AS old,
        (Flight_Volume - old)*100 / old AS Percentage_Change,
        Flight_Volume - old as Absolute_Change
        FROM df 
        GROUP BY Month
        ORDER BY Month ASC""").df()
# Drop the old axis, unneeded
df_ans.drop('old', axis = 1, inplace = True)
print("Overall flight volume, by month")
print(df_ans)
print(f"Standard Deviation of Percentage of Flights Delayed: {df_ans['Percentage_Of_Flights_Delayed'].std()}\nStandard Deviation of Percentage Change: {df_ans['Percentage_Change'].std()}%\nStandard Deviation of Absolute Change: {df_ans['Absolute_Change'].std()}")

Overall flight volume, by month
    Month Month_Name  Flight_Volume  Flight_Delayed  \
0       1    January         469968           95951   
1       2   February         429191           95179   
2       3      March         504312           95452   
3       4      April         485151           82247   
4       5        May         496993           89645   
5       6       June         503897          115742   
6       7       July         520718          107627   
7       8     August         510536           94113   
8       9  September         464946           60061   
9      10    October         486165           60079   
10     11   November         467972           70571   
11     12   December         479230           96772   

    Percentage_Of_Flights_Delayed  Percentage_Change  Absolute_Change  
0                       20.416496                NaN             <NA>  
1                       22.176374          -8.676548           -40777  
2                       18.927172   

Question 3b: What about for flights leaving from Boston (BOS) specifically?

In [8]:
# SQL Statement
df_ans = duckdb.sql("""
        SELECT MONTH as Month, monthname(make_date(2015, MONTH, ANY_VALUE(DAY))) as Month_Name, COUNT(FLIGHT_NUMBER) AS Flight_Volume,
        COUNT(AIR_SYSTEM_DELAY) AS Flight_Delayed, Flight_Delayed*100/Flight_Volume as Percentage_Of_Flights_Delayed,
        LAG(Flight_Volume) OVER (ORDER BY Month) AS old,
        (Flight_Volume - old)*100 / old AS Percentage_Change,
        Flight_Volume - old as Absolute_Change
        FROM df
        WHERE ORIGIN_AIRPORT = 'BOS'
        GROUP BY Month
        ORDER BY Month ASC""").df()
# Drop the old axis, unneeded
df_ans.drop('old', axis = 1, inplace = True)
print("Overall flight volume, by month")
print(df_ans)
print(f"Standard Deviation of Percentage of Flights Delayed: {df_ans['Percentage_Of_Flights_Delayed'].std()}\nStandard Deviation of Percentage Change: {df_ans['Percentage_Change'].std()}%\nStandard Deviation of Absolute Change: {df_ans['Absolute_Change'].std()}")

Overall flight volume, by month
    Month Month_Name  Flight_Volume  Flight_Delayed  \
0       1    January           8837            1939   
1       2   February           8380            2840   
2       3      March           9971            2405   
3       4      April          10000            1737   
4       5        May          10241            1688   
5       6       June          10544            2288   
6       7       July          10837            2119   
7       8     August          10727            2149   
8       9  September           9726            1213   
9      11   November           9464            1096   
10     12   December           9120            1889   

    Percentage_Of_Flights_Delayed  Percentage_Change  Absolute_Change  
0                       21.941835                NaN             <NA>  
1                       33.890215          -5.171438             -457  
2                       24.119948          18.985680             1591  
3                  

Question 4a: How many flights were cancelled in 2015?

In [9]:
print("Overall cancellation stats:")
# SQL Statement
df_ans = duckdb.sql("""SELECT df2.AIRLINE as Airline_Full_Name,
                        CASE WHEN df1.CANCELLATION_REASON IS NULL THEN 'Not Cancelled'
                        ELSE df3.CANCELLATION_DESCRIPTION END AS CANCELLATION_DESCRIPTION,
                        COUNT(df1.CANCELLATION_REASON) AS Flight_Count
                    FROM df1
                    LEFT JOIN
                    df2 ON df1.AIRLINE = df2.IATA_CODE
                    LEFT JOIN
                    df3 on df1.CANCELLATION_REASON = df3.CANCELLATION_REASON
                    GROUP BY df1.AIRLINE, df2.AIRLINE, df3.CANCELLATION_DESCRIPTION, df1.CANCELLATION_REASON
                    ORDER BY Airline_Full_Name, CANCELLATION_DESCRIPTION, Flight_Count
                    """).df()
print(df_ans)
print("How many flights were cancelled in 2015?")
# SQL Statement
df_ans = duckdb.sql("""SELECT COUNT(*) as Flight_Volume, COUNT(CANCELLATION_REASON) AS Flight_Cancelled_Count,
                    Flight_Cancelled_Count*100/Flight_Volume as Percentage_Cancelled
                    FROM df1
                    """).df()
print(df_ans)

Overall cancellation stats:
               Airline_Full_Name CANCELLATION_DESCRIPTION  Flight_Count
0           Alaska Airlines Inc.          Airline/Carrier           334
1           Alaska Airlines Inc.      National Air System            18
2           Alaska Airlines Inc.            Not Cancelled             0
3           Alaska Airlines Inc.                  Weather           317
4         American Airlines Inc.          Airline/Carrier          2879
5         American Airlines Inc.      National Air System           730
6         American Airlines Inc.            Not Cancelled             0
7         American Airlines Inc.                 Security             4
8         American Airlines Inc.                  Weather          7306
9   American Eagle Airlines Inc.          Airline/Carrier          2475
10  American Eagle Airlines Inc.      National Air System          3385
11  American Eagle Airlines Inc.            Not Cancelled             0
12  American Eagle Airlines Inc.    

Question 4b: What % of cancellations were due to weather? What % were due to the Airline/Carrier?

In [16]:
# SQL Statement
df_ans = duckdb.sql("""SELECT df2.AIRLINE as Airline_Full_Name,
                    COUNT(df1.CANCELLATION_REASON) AS Cancelled_Flight_Count,
                    SUM(CASE WHEN df1.CANCELLATION_REASON = 'B' THEN 1 ELSE 0 END) AS Flights_Cancelled_Due_To_Weather,
                    Flights_Cancelled_Due_To_Weather*100 / Cancelled_Flight_Count AS Percentage_Cancelled_Due_To_Weather,
                    SUM(CASE WHEN df1.CANCELLATION_REASON = 'A' THEN 1 ELSE 0 END) AS Flights_Cancelled_Due_To_Airline,
                    Flights_Cancelled_Due_To_Airline*100 / Cancelled_Flight_Count AS Percentage_Cancelled_Due_To_Airline
                    FROM df1
                    LEFT JOIN
                    df2 ON df1.AIRLINE = df2.IATA_CODE
                    GROUP BY df1.AIRLINE, df2.AIRLINE
                    ORDER BY Airline_Full_Name
                    """).df()
print(df_ans)
print(f'Flights cancelled due to weather: {df_ans['Flights_Cancelled_Due_To_Weather'].sum() } ')
print(f'Flights cancelled due to airlines: {df_ans['Flights_Cancelled_Due_To_Airline'].sum()}')
print(f'Flights cancelled due to weather (%): {df_ans['Flights_Cancelled_Due_To_Weather'].sum()*100/ df_ans['Cancelled_Flight_Count'].sum()}')
print(f'Flights cancelled due to airlines (%): {df_ans['Flights_Cancelled_Due_To_Airline'].sum()*100 / df_ans['Cancelled_Flight_Count'].sum()}')

               Airline_Full_Name  Cancelled_Flight_Count  \
0           Alaska Airlines Inc.                     669   
1         American Airlines Inc.                   10919   
2   American Eagle Airlines Inc.                   15025   
3    Atlantic Southeast Airlines                   15231   
4           Delta Air Lines Inc.                    3824   
5         Frontier Airlines Inc.                     588   
6         Hawaiian Airlines Inc.                     171   
7                JetBlue Airways                    4276   
8          Skywest Airlines Inc.                    9960   
9         Southwest Airlines Co.                   16043   
10              Spirit Air Lines                    2004   
11               US Airways Inc.                    4067   
12         United Air Lines Inc.                    6573   
13                Virgin America                     534   

    Flights_Cancelled_Due_To_Weather  Percentage_Cancelled_Due_To_Weather  \
0                     

Question 5: Which airlines seem to be most and least reliable, in terms of on-time departure?

In [11]:
# SQL Statement
df_ans = duckdb.sql("""SELECT df2.AIRLINE AS Airline_Full_Name, AVG(df1.DEPARTURE_DELAY) AS Departure_Delay
                    FROM df1
                    LEFT JOIN
                    df2 ON df1.AIRLINE = df2.IATA_CODE
                    WHERE DEPARTURE_DELAY IS NOT NULL
                    GROUP BY Airline_Full_Name
                    ORDER BY Departure_Delay""").df()
print("Top three most reliable airlines:")
print(df_ans.head(3))
print("Top three most unreliable airlines:")
print(df_ans.tail(3))

Top three most reliable airlines:
        Airline_Full_Name  Departure_Delay
0  Hawaiian Airlines Inc.         0.485713
1    Alaska Airlines Inc.         1.785801
2         US Airways Inc.         6.141137
Top three most unreliable airlines:
         Airline_Full_Name  Departure_Delay
11  Frontier Airlines Inc.        13.350858
12   United Air Lines Inc.        14.435441
13        Spirit Air Lines        15.944766
